# 축제 키워드 이항 변수화 과정
- 크롤링
- 자연어 처리
- 이항변수화

## 1.축제 키워드 크롤링

### A.축제 키워드 크롤링 - 전처리

In [2]:
import requests as rq
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from konlpy.corpus import kolaw
from konlpy.tag import Kkma
import nltk
import konlpy
from bs4 import BeautifulSoup as bs

In [6]:
base_festival = pd.read_excel("festival_info_1516_final.xlsx")

In [8]:
target_festival = pd.read_excel("festival_crawling_2019.xlsx")

#### 크롤링 과정
- 축제명을 네이버 뉴스에 검색
- 상위 5개 뉴스 헤드라인과 미리보기 문단을 크롤링
- 데이터 프레임을 새로 만들어 축제명을 키로 하여 관련 글귀를 저장

In [8]:
base_festival["축제명_수정"]

0               서울국제빵과자페스티벌
1                 아르헨티나페스티발
2                      한강홀릭
3                그랜드케이팝페스티벌
4                   서리풀페스티벌
              ...          
71               세종탄신기념문화행사
72               희망의운동화나눔축제
73                     빗물축제
74    남산골한옥마을세시맞이오(五)대감설날잔치
75                     종묘대제
Name: 축제명_수정, Length: 76, dtype: object

#### 2019년 데이터의 경우 축제명 끝에 2019가 달려있었기 때문에 이를 제거해주는 작업을 해줌

In [9]:
target_festival["title"]

0                  대한민국 산업기술 R&D대전 2019
1                        서울디자인페스티벌 2019
2                            서울무용제 2019
3                         서울지식이음축제 2019
4      코리아세일페스타 (Korea Sale FESTA) 2019
                     ...               
232                   서울 아시테지 겨울축제 2019
233                    서울 살롱 뒤 쇼콜라 2019
234                        서울영상광고제 2019
235                        서울영상광고제 2018
236                      하늘공원해맞이행사 2019
Name: title, Length: 237, dtype: object

In [10]:
target_festival["축제명_수정"]=target_festival["title"].map(lambda x : x[:-4])

### B.축제 키워드 크롤링 - 불러오기

#### 2015,2016년 크롤링

In [11]:
target_festival

,Unnamed: 0,title,img,개최지역,개최기간,축제성격,관련 누리집,축제장소,요금,소요시간,...,explanation,오시는 길,부대행사,축제장소_수정,축제명_수정,address_name,x,y,road_address_name,place_url
0,0,대한민국 산업기술 R&D대전 2019,"<img alt=""대한민국 산업기술 R&amp;D대전 2019 관련 이미지 "" sr...",서울시 강남구,2019.12.13~2019.12.14,기타,http://www.ktechshow.or.kr,서울 코엑스 1층 Hall B,무료,기간 내 자유,...,"\n\n\t\t\t\t\t\n\t\t\t\t\t산업기술의 꿈, 당신의 미래! Ind...",NaN,NaN,코엑스,대한민국 산업기술 R&D대전,NaN,127.056787,37.512370,서울 강남구 영동대로 513,https://place.map.kakao.com/17573702
1,1,서울디자인페스티벌 2019,"<img alt=""서울디자인페스티벌 2019 관련 이미지 "" src=""/attach...",서울시 강남구,2019. 12. 4. ~ 8,문화예술,http://seoul.designfestival.co.kr,Coex 3층 Hall C,"슈퍼 얼리버드 8,000원 / 얼리버드 10,000원 / 현장구매 12,000원",기간 내 자유,...,\n\n\t\t\t\t\t\n\t\t\t\t\t서울디자인페스티벌은 1976년부터 축...,NaN,NaN,코엑스,서울디자인페스티벌,NaN,NaN,NaN,NaN,NaN
2,2,서울무용제 2019,"<img alt=""서울무용제 2019 관련 이미지 "" src=""/attachFile...",서울시 종로구,2019. 10. 12. ` 11. 29.,문화예술,http://sdf1979.koreadanceassociation.org,"아르코예술극장 대극장/이화여자대학교 삼성홀/강동아트센터 소극장 드림, 상명아트센터 대신홀","공연별 상이함 (VIP석 50,000원 / R석 30,000원 / S석 20,000...",약 90분,...,\n\n\t\t\t\t\t\n\t\t\t\t\t「서울무용제」 는 우수한 창작무용 공...,"* 지하철\n\n4호선 혜화역 하차(2번출구) 하차 문예진흥원, 마로니에공원 방향\...",NaN,아르코예술극장 대극장/이화여자대학교 삼성홀/강동아트센터 소극장 드림,서울무용제,NaN,NaN,NaN,NaN,NaN
3,3,서울지식이음축제 2019,"<img alt=""서울지식이음축제 2019 관련 이미지 "" src=""/attachF...",서울시 중구,2019.11.23. ~ 2019.11.24.,문화예술,http://www.seoul-ieum.kr/forum/intro.php,서울도서관,무료,기간 내 자유,...,\n\n\t\t\t\t\t\n\t\t\t\t\t(舊)서울 북 페스티벌이 2019년...,NaN,NaN,서울도서관,서울지식이음축제,서울 강남구 삼성동 28,127.046979,37.513577,NaN,http://place.map.kakao.com/1150117864
4,4,코리아세일페스타 (Korea Sale FESTA) 2019,"<img alt=""코리아세일페스타 (Korea Sale FESTA) 2019 관련 ...",서울시 중구,2019. 11. 1. ~ 22.,기타,http://www.koreasalefesta.kr,대한민국 전역,NaN,NaN,...,\n\n\t\t\t\t\t\n\t\t\t\t\t코리아세일페스타(KOREA Sale ...,NaN,NaN,대한민국 전역,코리아세일페스타 (Korea Sale FESTA),NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,232,서울 아시테지 겨울축제 2019,"<img alt=""서울 아시테지 겨울축제 2019 관련 이미지 "" src=""/att...",서울시 종로구,2019.01.09(수)~01.19(토) / 공연별로 상이,문화예술,http://www.assitejkorea.org,올림픽기념국민생활관 신관 아이들극장,"전석 20,000원 / [할인정보] 3인이상(30% 할인) / 20인 이상 (단체할...",공연마다 상이,...,\n\n\t\t\t\t\t\n\t\t\t\t\t[행사소개]\n\n(사)국제아동청소년...,NaN,[행사내용]\n- 겨울축제 개막작인 <깔깔나무>는 아시아문화원이 제작한 인형음악극으...,올림픽기념국민생활관 신관 아이들극장,서울 아시테지 겨울축제,NaN,NaN,NaN,NaN,NaN
233,233,서울 살롱 뒤 쇼콜라 2019,"<img alt=""서울 살롱 뒤 쇼콜라 2019 관련 이미지 "" src=""/atta...",서울시 강남구,2019. 01. 10 ~ 01. 13,기타,http://www.salonduchocolat.kr,서울 코엑스 Hall B,"10,000원",기간내 자유,...,\n\n\t\t\t\t\t\n\t\t\t\t\t전세계 950만이 선택한 나를 위한 ...,"[자동차 이용시]\n\n1. 강남역 방면\n테헤란로 삼성동 방면으로 직진, 삼성역 ...",- 초콜릿과 패션이 결합된 초콜릿 패션쇼(Chocolate Fashion Show)...,코엑스,서울 살롱 뒤 쇼콜라,NaN,NaN,NaN,NaN,NaN
234,234,서울영상광고제 2019,"<img alt=""서울영상광고제 2019 관련 이미지 "" src=""/attachFi...",서울시 중구,2019. 1. 3. / 17:00 ~19:00,문화예술,http://www.tvcfaward.com,서울특별시청 8층 다목적홀,"유료 예약시 : 5,000원, 무료 예약시 : 추첨홈페이지 내 사전 관람 접수",2시간,...,\n\n\t\t\t\t\t\n\t\t\t\t\t[행사소개]\n광고영상제의 불모지였던...,NaN,"[행사내용]\n심사 결과 발표 (그랑프리/금/은/동/특별상), 아티스트 초청 공연",서울특별시청 8층 다목적홀,서울영상광고제,NaN,NaN,NaN,NaN,NaN
235,235,서울영상광고제 2018,"<img alt=""서울영상광고제 2018 관련 이미지 "" src=""/attachFi...",서울시 중구,17:00 ~19:00,문화예술,http://www.tvcfaward.com,서울특별시청 8층 다목적홀,"유료 예약시 : 5,000원, 무료 예약시 : 추첨홈페이지 내 사전 관람 접수",2시간,...,\n\n\t\t\t\t\t\n\t\t\t\t\t[행사소개]\n광고 영상제의 불모지였...,NaN,"[행사내용]\n심사 결과 발표 (그랑프리/금/은/동/특별상), 아티스트 초청 공연",서울특별시청 8층 다목적홀,서울영상광고제,NaN,NaN,NaN,NaN,NaN


In [12]:
base_festag = pd.DataFrame()

In [13]:
for j in range(len(base_festival["축제명_수정"])) :
    url = "https://search.naver.com/search.naver?where=news&sm=tab_jum&query=" + str(base_festival["축제명_수정"][j])
    res = rq.get(url=url)
    soup = bs(res.text,"html.parser")
    test = []
    for i in range(10) :
        temp = soup.select("#sp_nws"+str(i)+" > dl > dd:nth-child(3)")
        test.append(str(temp))
    base_festag = base_festag.append(pd.DataFrame(np.array([base_festival["축제명_수정"][j],test]).reshape(1,-1)))
    time.sleep(1)

In [20]:
base_festag

,0,1
0,서울국제빵과자페스티벌,"[[], [<dd> 백운용 기자 = 대구공업대학교(총장 이별나) 호텔외식조리계열 창..."
0,아르헨티나페스티발,"[[], [<dd>18일(현지시간) 오후 프랑스 칸 팔레 드 <strong clas..."
0,한강홀릭,"[[], [<dd><strong class=""hl"">한강</strong>에서 최근 ..."
0,그랜드케이팝페스티벌,"[[], [<dd>인터콘티넨탈호텔 <strong class=""hl"">그랜드</str..."
0,서리풀페스티벌,"[[], [<dd>이와 관련, 전희경 한국당 대변인은 전날 페이스북에 서초구청에서 ..."
...,...,...
0,세종탄신기념문화행사,"[[], [<dd>송시열선생 <strong class=""hl"">탄신</strong>..."
0,희망의운동화나눔축제,"[[], [<dd><strong class=""hl"">희망의 운동화 나눔축제</str..."
0,빗물축제,"[[], [<dd>이번 박람회 주제는 '내가 그린 건강한 세상, 착한소비 <stro..."
0,남산골한옥마을세시맞이오(五)대감설날잔치,"[[], [], [], [], [], [], [], [], [], []]"


In [22]:
base_festag.to_csv("fes1516_tag.csv")

#### 2019년 크롤링

In [94]:
target_festag = pd.DataFrame()

In [95]:
len(target_festival["축제명_수정"])

237

In [96]:
for j in range(len(target_festival["축제명_수정"])) :
    url = "https://search.naver.com/search.naver?where=news&sm=tab_jum&query=" + str(target_festival["축제명_수정"][j])
    res = rq.get(url=url)
    soup = bs(res.text,"html.parser")
    test = []
    for i in range(10) :
        temp = soup.select("#sp_nws"+str(i)+" > dl > dd:nth-child(3)")
        test.append(str(temp))
    target_festag = target_festag.append(pd.DataFrame(np.array([target_festival["축제명_수정"][j],test]).reshape(1,-1)))
    time.sleep(1)

In [97]:
target_festag

,0,1
0,대한민국 산업기술 R&D대전,"[[], [<dd><strong class=""hl"">R</strong>&amp;D)..."
0,서울디자인페스티벌,"[[], [<dd>예술경영지원센터가 ‘아트 굿즈 편의점’을 국내 디자이너들의 세계 ..."
0,서울무용제,"[[], [<dd>■ <strong class=""hl"">서울무용제</strong> ..."
0,서울지식이음축제,"[[], [<dd>2019 <strong class=""hl"">서울지식이음축제</st..."
0,코리아세일페스타 (Korea Sale FESTA),"[[], [<dd>(사진=LG전자) LG전자가 다음달 1일부터 22일까지 진행하는 ..."
...,...,...
0,서울 아시테지 겨울축제,"[[], [<dd>한편 종로구는 오는 2020년 1월 8일부터 19일까지 종로 아이..."
0,서울 살롱 뒤 쇼콜라,"[[], [<dd>한지웅 영상기자 엄선영 인턴기자 = 10일 서울 강남구 코엑스에서..."
0,서울영상광고제,"[[], [<dd>뮤지컬 '라이온 킹' 인터내셔널 투어 한국 공연을 기념해 롯데월드..."
0,서울영상광고제,"[[], [<dd>뮤지컬 '라이온 킹' 인터내셔널 투어 한국 공연을 기념해 롯데월드..."


In [ ]:
target_festag.to_csv("fes19_tag.csv")

## 2.자연어 처리

### A.뉴스 키워드 자연어 처리

In [21]:
kkma = Kkma()

C:\Users\User\AppData\Local\Continuum\anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [26]:
len(base_festag)

76

In [154]:
base_festag = pd.read_csv("fes1516_tag.csv").iloc[:,1:]

In [155]:
base_festag.columns = ["축제명","축제주요내용"]

In [157]:
base_festag.index= range(len(base_festag))

In [158]:
base_festag_nouns = base_festag

In [162]:
for i in range(len(base_festag)) :
    container = base_festag["축제주요내용"][i]
    base_festag_nouns["축제주요내용"][i] = kkma.nouns(container)

In [163]:
base_festag_nouns.to_csv("fes_tag_nouns1516.csv")

In [164]:
target_festag = pd.read_csv("fes19_tag.csv").iloc[:,1:]

In [165]:
target_festag.columns = ["축제명","축제주요내용"]

In [166]:
target_festag.index= range(len(target_festag))

In [167]:
target_festag_nouns = target_festag

In [169]:
for i in range(len( target_festag)) :
    container =  target_festag["축제주요내용"][i]
    target_festag_nouns["축제주요내용"][i] = kkma.nouns(container)

In [171]:
target_festag_nouns

,축제명,축제주요내용
0,대한민국 산업기술 R&D대전,"[사업, 등, 적극적, 지원, 저, 3, 3차, 차, 지능, 로봇, 기본, 기본계획..."
1,서울디자인페스티벌,"[예술, 예술경영지원센터, 경영, 지원, 센터, 아트, 굿, 굿즈, 즈, 편의점, ..."
2,서울무용제,"[서울, 용제, 2019, 서울특별, 서울특별시, 특별시, 시, 종로, 종로구, 구..."
3,서울지식이음축제,"[2019, 서울, 서울지식, 지식, 음, 음축제, 축제, 열린, 24, 24일, ..."
4,코리아세일페스타 (Korea Sale FESTA),"[사진, 전자, 다음달, 1, 1일, 일, 22, 22일, 진행, 코리아, 코리아세..."
...,...,...
232,서울 아시테지 겨울축제,"[한편, 0, 0종로구, 종로, 종로구, 구, 0오, 오, 02020, 02020년..."
233,서울 살롱 뒤 쇼콜라,"[웅, 영상, 영상기자, 기자, 엄선, 엄선영, 영, 인턴, 인턴기자, 10, 10..."
234,서울영상광고제,"[뮤지컬, 라이온, 킹, 인터, 인터내셔널, 내셔널, 투어, 한국, 공연, 기념, ..."
235,서울영상광고제,"[뮤지컬, 라이온, 킹, 인터, 인터내셔널, 내셔널, 투어, 한국, 공연, 기념, ..."


### B.축제설명 자연어 처리

In [45]:
target_festag_nouns = pd.read_csv("fes_tag_nouns19.csv").iloc[:,1:]
base_festag_nouns = pd.read_csv("fes_tag_nouns1516.csv").iloc[:,1:]

In [23]:
base_explain_nouns = base_festival[["축제명_수정","축제설명_수정"]]
for i in range(len( base_festival)) :
    container =  base_festival["축제설명_수정"][i]
    base_explain_nouns["축제설명_수정"][i] = kkma.nouns(container)

C:\Users\User\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [72]:
base_festag_nouns["축제주요내용"] = base_festag_nouns["축제주요내용"].map(lambda x : x[1:-1].split("'"))

In [95]:
base_noun = pd.DataFrame(base_festag_nouns["축제명"])

In [97]:
base_noun["축제주요내용"] = pd.DataFrame(np.array([i+j for i,j in zip(base_explain_nouns.iloc[:,1],base_festag_nouns.iloc[:,1])]))

In [100]:
base_noun = base_noun.iloc[:-1,:]

In [105]:
target_noun = pd.DataFrame(target_festag_nouns["축제명"])

In [116]:
target_noun["축제주요내용"] = pd.DataFrame(np.array([i+j for i,j in zip(target_explain_nouns.iloc[:,1],target_festag_nouns.iloc[:,1])]))

In [117]:
target_noun

,축제명,축제주요내용
0,대한민국 산업기술 R&D대전,"[산업, 산업기술, 기술, 꿈, 당신, 미래, 2019, 대한, 대한민국, 민국, ..."
1,서울디자인페스티벌,"[서울, 서울디자인페스티벌, 디자인, 페스티벌, 1976, 1976년, 년, 축적,..."
2,서울무용제,"[서울, 용제, 우수, 창작, 창작무용, 무용, 공연, 대한, 대한민국, 민국, 무..."
3,서울지식이음축제,"[서울, 북, 페스티벌, 2019, 2019년, 년, 서울지식, 지식, 음, 음포럼..."
4,코리아세일페스타 (Korea Sale FESTA),"[코리아, 코리아세일페스타, 세일, 페, 스타, 제조, 유통, 분야, 다양, 기업,..."
...,...,...
232,서울 아시테지 겨울축제,"[행사, 행사소개, 소개, 사, 국제, 국제아동청소년연극, 아동, 청소년, 연극, ..."
233,서울 살롱 뒤 쇼콜라,"[전세계, 950, 950만이, 만이, 선택, 나, 선물, 살롱, 뒤, 쇼, 쇼콜라..."
234,서울영상광고제,"[행사, 행사소개, 소개, 광고, 광고영상제, 영, 상제, 불모지, 한국, 뿌리, ..."
235,서울영상광고제,"[행사, 행사소개, 소개, 광고, 영상, 영상제의, 제의, 불모지, 한국, 뿌리, ..."


### C. 축제 키워드 통계

In [144]:
tag_stat19 = []
for i in range(len(target_noun)) :
    for j in target_noun["축제주요내용"][i] :
        tag_stat19.append(j)

In [145]:
 tag_stat19 =pd.DataFrame(tag_stat19)

In [146]:
 tag_stat19.columns=["keyword"]

In [147]:
tag_stat19 = pd.DataFrame(tag_stat19.groupby("keyword")["keyword"].count())

In [150]:
tag_stat19.columns=["빈도"]

In [154]:
tag_stat19 = pd.DataFrame(tag_stat19).sort_values(by="빈도",ascending=False)

In [12]:
tag_stat19.to_csv("tag_stat19.csv",encoding="ANSI")

In [159]:
tag_stat1516 = []
for i in range(len(base_noun)) :
    for j in base_noun["축제주요내용"][i] :
        tag_stat1516.append(j)

In [161]:
 tag_stat1516 =pd.DataFrame(tag_stat1516)

In [163]:
 tag_stat1516.columns=["keyword"]

In [164]:
tag_stat1516 = pd.DataFrame(tag_stat1516.groupby("keyword")["keyword"].count())

In [165]:
 tag_stat1516.columns=["빈도"]

In [166]:
tag_stat1516 = pd.DataFrame(tag_stat1516).sort_values(by="빈도",ascending=False)

In [13]:
tag_stat1516.to_csv("tag_stat1516.csv",encoding="ANSI")

나머지는 엑셀에서 수작업하여 키워드를 쳐냄
- 빈도가 너무 작은 키워드는 쳐냄
- 부적절해 보이는 키워드는 쳐냄

In [20]:
sanwi1516 = pd.read_csv("sanwi1516.csv",encoding="ANSI").dropna()

In [21]:
sanwi19 = pd.read_csv("sanwi19.csv",encoding="ANSI").dropna()

In [38]:
keyword_index = [i in [i for i in sanwi1516.iloc[:,0]] for i in sanwi19.iloc[:,0]]

In [43]:
keyword = sanwi19.iloc[keyword_index,0]

In [44]:
keyword.head()

0    축제
1    서울
2    개최
3    문화
4    행사
Name: keyword, dtype: object

## 3. 이항변수화

In [89]:
target_tag = []
for i in keyword :
    for j in target_festag_nouns["축제주요내용"] :
        target_tag.append(i in j)

In [90]:
target_tag= pd.DataFrame(np.transpose(np.array([target_tag[len(target_festag_nouns)*i:len(target_festag_nouns)*(1+i)] for i in range(len(keyword))])))
target_tag.columns = np.array(keyword).flatten()
target_tag = pd.merge(target_festag_nouns["축제명"],target_tag.applymap(int),left_index=True,right_index=True)

In [91]:
target_tag

,축제명,축제,서울,개최,문화,행사,다양,공연,진행,체험,...,도시,이벤트,전국,주민,한강,작품,대회,주관,어린이,기업
0,대한민국 산업기술 R&D대전,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,서울디자인페스티벌,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,서울무용제,0,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,서울지식이음축제,1,1,0,1,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,코리아세일페스타 (Korea Sale FESTA),0,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,서울 아시테지 겨울축제,1,1,1,1,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
233,서울 살롱 뒤 쇼콜라,1,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
234,서울영상광고제,0,1,1,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
235,서울영상광고제,0,1,1,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0


In [85]:
base_tag = []
for i in keyword :
    for j in base_festag_nouns["축제주요내용"] :
        base_tag.append(i in j)

In [86]:
base_tag= pd.DataFrame(np.transpose(np.array([base_tag[len(base_festag_nouns)*i:len(base_festag_nouns)*(1+i)] for i in range(len(keyword))])))
base_tag.columns = np.array(keyword).flatten()
base_tag = pd.merge(base_festag_nouns["축제명"],base_tag.applymap(int),left_index=True,right_index=True)

In [87]:
base_tag

,축제명,축제,서울,개최,문화,행사,다양,공연,진행,체험,...,도시,이벤트,전국,주민,한강,작품,대회,주관,어린이,기업
0,서울국제빵과자페스티벌,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,아르헨티나페스티발,0,0,1,0,1,0,1,1,0,...,1,0,0,0,0,0,0,0,0,0
2,한강홀릭,1,1,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,그랜드케이팝페스티벌,0,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,서리풀페스티벌,1,1,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,세종탄신기념문화행사,0,0,0,1,1,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
72,희망의운동화나눔축제,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
73,빗물축제,1,1,1,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,1
74,남산골한옥마을세시맞이오(五)대감설날잔치,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [92]:
target_tag.to_csv("target_tag.csv")
base_tag.to_csv("base_tag.csv")

## 서비스에 띄워줄 태그들 선별

In [5]:
target_tag = pd.read_csv("target_tag.csv")
target_festag_nouns = pd.read_csv("fes_tag_nouns19.csv").iloc[:,1:]

In [70]:
target_tag.columns

Index(['Unnamed: 0', '축제명', '축제', '서울', '개최', '문화', '행사', '다양', '공연', '진행',
       '체험', '페스티벌', '시민', '참여', '프로그램', '한국', '예술', '일대', '제공', '대표', '공원',
       '주제', '국내', '세계', '오후', '우리', '지역', '대한민국', '광장', '사진', '주최', '거리',
       '구청장', '전시', '전통', '기자', '무대', '역사', '음악', '가족', '도시', '이벤트', '전국',
       '주민', '한강', '작품', '대회', '주관', '어린이', '기업'],
      dtype='object')

In [25]:
tag_selection =pd.DataFrame(np.zeros(shape=(len(target_festag_nouns.iloc[:,1]),len(target_tag.columns[1:]))))

In [67]:
for i in range(1,len(target_tag.columns)) :
    for j in range(len(target_festag_nouns.iloc[:,1])) :
        tag_selection.iloc[j,i-1] = sum([target_tag.columns[i] in k for k in target_festag_nouns.iloc[j,1].split(",")])


In [71]:
tag_selection.columns = target_tag.columns[1:]

In [75]:
tag_selection.iloc[:,0] = target_festag_nouns["축제명"]

In [105]:
main_keyword = pd.DataFrame()

In [109]:
pd.DataFrame(tag_selection.iloc[i,1:]).sort_values(by = 0,ascending=False).index[:5]

Index(['한국', '전시', '주최', '세계', '국내'], dtype='object')

In [128]:
main_keyword  = []
for i in range(len(tag_selection)) :
    main_keyword.append([i for i in tag_selection.iloc[i,1:].sort_values(ascending=False).index[:7]])

In [129]:
main_keyword = pd.DataFrame(main_keyword)

In [130]:
main_keyword.index = target_festag_nouns["축제명"]

In [132]:
main_keyword

,0,1,2,3,4,5,6
축제명,,,,,,,
대한민국 산업기술 R&D대전,한국,국내,대한민국,행사,주최,전시,세계
서울디자인페스티벌,페스티벌,기업,서울,국내,예술,세계,참여
서울무용제,예술,한국,서울,대한민국,문화,공연,주최
서울지식이음축제,서울,축제,광장,공원,문화,역사,시민
코리아세일페스타 (Korea Sale FESTA),행사,참여,서울,사진,진행,한국,국내
...,...,...,...,...,...,...,...
서울 아시테지 겨울축제,예술,공연,축제,한국,서울,국내,대한민국
서울 살롱 뒤 쇼콜라,기자,축제,행사,주제,오후,세계,서울
서울영상광고제,한국,서울,개최,한강,대한민국,공연,주최


In [133]:
main_keyword.to_csv("서비스용_키워드_선별")

In [134]:
main_keyword

,0,1,2,3,4,5,6
축제명,,,,,,,
대한민국 산업기술 R&D대전,한국,국내,대한민국,행사,주최,전시,세계
서울디자인페스티벌,페스티벌,기업,서울,국내,예술,세계,참여
서울무용제,예술,한국,서울,대한민국,문화,공연,주최
서울지식이음축제,서울,축제,광장,공원,문화,역사,시민
코리아세일페스타 (Korea Sale FESTA),행사,참여,서울,사진,진행,한국,국내
...,...,...,...,...,...,...,...
서울 아시테지 겨울축제,예술,공연,축제,한국,서울,국내,대한민국
서울 살롱 뒤 쇼콜라,기자,축제,행사,주제,오후,세계,서울
서울영상광고제,한국,서울,개최,한강,대한민국,공연,주최
